In [4]:
import numpy as np
import pandas as pd

# Draft Elips

In [ ]:
#    t1 t2 .. tn
# l1 xx xx    xx r1
# l2 xx xx    xx r2
# ..
# lm xx xx    xx rm
#    b1 b2    bn

# row => m
# col => n

In [12]:
m = 1
n = 2

# kalo nilainya sama pake ini
t = np.full(n, 5)
r = np.full(m, 55)
b = np.full(n, 25)
l = np.full(m, 30)

In [14]:
matrix = np.zeros((m+2, n+2))
matrix

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

# Parabola

In [37]:
# HELPER FUNCTIONS | gunakan jika perlu
def getAlpha(k, rho, C):
    return k/(rho*C)


def getLambda(alpha, dt, dx):
    return alpha*dt/(dx**2)


def getJ(t, dt):
    return int(round(t/dt))

In [94]:
# misal kasus (di buku chapra halaman 882)
# 100 ? ? ? ? 50 (suhu mula2 0)

# env = suhu lingkungan (suhu awal) = 0
# l   = suhu kiri        = 100
# r   = suhu kanan       = 50
# I   = ? ada ...        = 4
# J   = sampai j ke-...  = 10 (delta t = 1s -> selama 10s -> berarti sampai j10)
# q   = lambda           = 0.20875

## Metode Eksplisit

Menghitung masa depan dari masa lalu

In [45]:
def IterateExplicitParabol(Tj, j, I, J, q):
    Tjj = Tj.copy()

    for i in range(1, I+1):
        Tjj[i] = Tj[i] + q*(Tj[i-1] - 2*Tj[i] + Tj[i+1])

    if j < J:
        return IterateExplicitParabol(Tjj, j+1, I, J, q)

    return Tjj


def ExplicitParabol(env, l, r, I, J, q):
    Tj0 = np.concatenate((l, np.full(I, env, dtype="float32"), r), axis=None)

    return IterateExplicitParabol(Tj0, 1, I, J, q)


ExplicitParabol(env=0, l=100, r=50, I=4, J=10, q=0.20875)

array([100.      ,  65.91471 ,  41.481525,  32.019577,  36.7853  ,
        50.      ], dtype=float32)

## Metode Implisit

Menghitung masa lalu dari masa depan

In [44]:
def CreateMatrixImplicitParabol(T, q):
    I = T.size
    A = np.zeros((I, I))
    B = np.zeros(I)

    for i in range(1, I-1):
        B[i] += T[i]

        if i == 1:
            B[i] += q*T[i-1]
        else:
            A[i, i-1] = -q

        A[i, i] = 1 + 2*q

        if i+2 == I:
            B[i] += q*T[i+1]
        else:
            A[i, i+1] = -q

    return [A[1:I-1, 1:I-1], B[1:I-1]]


def IterateImplicitParabol(Tj, j, J, q):
    A, B = CreateMatrixImplicitParabol(Tj, q)
    X = np.linalg.solve(A, B)

    Tjj = np.concatenate((Tj[0], X, Tj[-1]), axis=None, dtype="float32")

    if j < J:
        return IterateImplicitParabol(Tjj, j+1, J, q)

    return Tjj


def ImplicitParabol(env, l, r, I, J, q):
    Tj0 = np.concatenate((l, np.full(I, env, dtype="float32"), r), axis=None, dtype="float32")

    return IterateImplicitParabol(Tj0, 1, J, q)


ImplicitParabol(env=0, l=100, r=50, I=4, J=10, q=0.20875)

array([100.      ,  63.490936,  38.759876,  30.02051 ,  35.58047 ,
        50.      ], dtype=float32)

## Metode Crank-Nicolson

Gabungan dari Eksplisit dan Implisit

In [43]:
def CreateMatrixCrankNicolson(T, q):
    I = T.size
    A = np.zeros((I, I))
    B = np.zeros(I)

    for i in range(1, I-1):
        B[i] += q*T[i-1] + 2*(1-q)*T[i] + q*T[i+1]

        if i == 1:
            B[i] += q*T[i-1]
        else:
            A[i, i-1] = -q

        A[i, i] = 2 + 2*q

        if i+2 == I:
            B[i] += q*T[i+1]
        else:
            A[i, i+1] = -q

    return [A[1:I-1, 1:I-1], B[1:I-1]]


def IterateCrankNicolson(Tj, j, J, q):
    A, B = CreateMatrixCrankNicolson(Tj, q)
    X = np.linalg.solve(A, B)

    Tjj = np.concatenate((Tj[0], X, Tj[-1]), axis=None, dtype="float32")

    if j < J:
        return IterateCrankNicolson(Tjj, j+1, J, q)

    return Tjj


def CrankNicolson(env, l, r, I, J, q):
    Tj0 = np.concatenate((l, np.full(I, env, dtype="float32"), r), axis=None, dtype="float32")

    return IterateCrankNicolson(Tj0, 1, J, q)


CrankNicolson(env=0, l=100, r=50, I=4, J=10, q=0.20875)

array([100.      ,  64.767075,  40.061157,  30.94472 ,  36.201607,
        50.      ], dtype=float32)